# Sistemas de filas

Após uma contagem de, pelo menos, uma hora em um sistema que apresente fila de espera, preferencialmente com mais de um servidor, o aluno deverá apresentar respostas às perguntas que serão apresentadas a seguir.

In [1]:
from datetime import datetime, timedelta
from IPython.display import display
from math import factorial
from matplotlib import pyplot as plt

import io
import numpy as np
import pandas as pd

Σ = sum

%matplotlib inline

### Dados

In [2]:
def timetable(a, b):
    return b + timedelta(minutes=int(a))
    
v_timetable = np.vectorize(timetable)

In [3]:
def sampling_generator(
    size: int=20, initial_datetime: datetime=None
) -> np.array:
    """
    
    """
    if initial_datetime is None:
        initial_datetime = datetime.now()
    
    return v_timetable(
        np.random.randint(0, 60, size=size),
        datetime.now()
    )

In [4]:
def mean_diff_time(se_time: pd.Series) -> float:
    """
    
    """
    n = se_time.shape[0]
    l_diff = [0] * (n-1)
    
    for i in range(1, n):
        j = i-1
        l_diff[j] = (se_time[i] - se_time[j]).seconds/60
    
    return np.mean(l_diff)

In [5]:
initial_datetime = datetime.now()

df_chegada = pd.DataFrame({
    'entrada': sampling_generator(
        initial_datetime=initial_datetime
    )
}).sort_values(by='entrada').reset_index(drop=True)

df_saida = pd.DataFrame({
    'saída': sampling_generator(
        initial_datetime=initial_datetime
    )
}).sort_values(by='saída').reset_index(drop=True)

"""
fig, ax = plt.subplots(1, 2)
df_chegada.hist(ax=ax[0])

df_saida.hist(ax=ax[1])
plt.show()
"""
print('tables head')
display(df_chegada.head())
display(df_saida.head())

tables head


,entrada
0,2016-06-09 12:26:58.817949
1,2016-06-09 12:28:58.817949
2,2016-06-09 12:32:58.817949
3,2016-06-09 12:34:58.817949
4,2016-06-09 12:40:58.817949


,saída
0,2016-06-09 12:15:58.912541
1,2016-06-09 12:20:58.912541
2,2016-06-09 12:21:58.912541
3,2016-06-09 12:22:58.912541
4,2016-06-09 12:26:58.912541


**a) Descrição do sistema de filas, local, data e horários da coleta de dados:**

Fila do tipo: FIFO

Local: loja Plazafone

Início da coleta:

In [6]:
initial_datetime.strftime('%d de %b de %Y - %l:%M%p')

'09 de Jun de 2016 - 12:15PM'

**b) Número de servidores atendendo = S:**

In [7]:
S = 3
print('S:', S)

S: 3


**c) O tempo médio entre as chegadas à fila ( um cliente a cada 5 min, por ex.):**

In [26]:
μ_chegada = mean_diff_time(df_chegada['entrada'])
print('μ_chegada:', μ_chegada, 'mins')

μ_chegada: 2.42105263158 mins


**d) A taxa de chegada de usuários no sistema de filas – λ ( clientes por unidade de tempo):**

$\lambda = \frac{60 min}{\mu\_chegada} (clientes/hora)$

In [9]:
λ = 60/μ_chegada
print('λ:', λ)

λ: 24.7826086957


**e) O tempo médio de atendimento de cada cliente ( 20 min, por ex.):**

In [25]:
μ_atendimento = mean_diff_time(df_saida['saída'])
print('μ_atendimento:', μ_atendimento, 'mins')

μ_atendimento: 2.94736842105 mins


**f) A taxa de atendimento dos usuários – μ ( clientes por unidade de tempo, por servidor:**

$\mu = \frac{60 min}{\mu\_atendimento m} (clientes/hora)$

Se existir mais que um servidor, a taxa de atendimento do sistema será multiplicada pelo

número de servidores : S=5, então a taxa será de 5μ=15 clientes por hora.

In [27]:
μ = 60/μ_atendimento
print('μ:', μ, 'clientes/hora')

μ: 20.3571428571 clientes/hora


**g) A ocupação do sistema(em porcentagem);**

$p = \frac{\lambda}{S*\mu} * 100 (\%)$

In [28]:
p = (λ/S*μ)*100
print('p:', p, '%')

p: 16816.7701863 %


h) A probabilidade de o sistema estar vazio;

$$
P_0 = [
    \sum\limits_{n=0}^{S-1} (\frac{(\lambda / \mu)^n}{n!}) + 
    \frac{(\lambda/\mu)^S}{S!(1-(\lambda/(S*\mu)))}
]^{-1}
$$

In [29]:
P_0 = Σ([
    (((λ/μ)**n)/(factorial(n))) +
    (((λ/μ)**S)/(factorial(S) * (1-(λ/(S*μ)))))
    for n in range(S)
])**(-1)

print('P_0:', P_0)

P_0: 0.22338376608


i) A probabilidade de todos os servidores estarem ocupados:

$
P_{ocupados} = P(n \leq S) = \frac{(\lambda/\mu)^S}{S!(1-\lambda/(S.\mu))}.P_0
$


In [15]:
P_ocupados = (((λ/μ)**S)/(factorial(S)*(μ*S-λ))) - P_0
print('P_ocupados:', P_ocupados)

P_ocupados: -0.215097354691


k) O número esperado de usuários na fila:

$
L_q = \frac{(\lambda/\mu)^S . \lambda . \mu . S}{S!(\mu . S - \lambda)} . P_0
$

In [19]:
L_q = ((((λ/μ)**S)*λ*μ*S)/(factorial(S)*((μ*S-λ)**2)))*P_0
print('L_q:', L_q)

L_q: 0.0772023202296


j) O número esperado de usuários no sistema:

$
L = L_q + \frac{\lambda}{\mu}
$

In [20]:
L = L_q + (λ/μ)
print('L:', L)

L: 1.29459362458


l) O tempo médio dos usuários na fila:

$
W_q = \frac{L_q}{\lambda}
$

In [23]:
W_q = L_q/λ
print('W_q:', W_q)

W_q: 0.0031151813426


m) O tempo provável dos usuários no sistema:

$
W = \frac{L}{\lambda}
$

In [24]:
W = L/λ
print('W:', W)

W: 0.0522379883601


n) Se a taxa de chamadas duplicasse, o que aconteceria com o sistema? E quais as providências que deveriam ser tomadas.

## Referências

http://www.mathpages.com/home/kmath026/kmath026.htm

http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.random.poisson.html
